# Analyze Maricopa County, Arizona  Biodiversity Intactness Index (BII) changes from 2017-2020

Author: Hazel Vaquero

Repository Link: https://github.com/hazelvaq/Maricopa-Arizona-Changes-in-Biodiversity-Intactness-Index-.git 

## About 

In this notebook we analyse the changes observed in Biodiversity Intactness Index (BII) in Maricopa County, Phoenix, Arizona from 2017 to 2020. In 2021, Maricopa County was reported as the fastest growing county in the United States since 2001. It is the 4th largest county in terms of population. Urban development creates pressure on biodiversity and natural areas. This notebook seeks to analyze the impact of the rapid urban expansion in Maricopa County. 

## About the Datasets

In this notebook we use two different datasets:

**1. BII Data** 

The first dataset is from the [Planetary Computer Impact Observatory](https://planetarycomputer.microsoft.com/dataset/io-biodiversity). This dataset estimates terrestrial Biodiversity Intactness as a 100-meter gridded maps for the years 2017-2020. Biodiversity intactness is an estimated combination of two metrics:
- Abundance
- Compositional Similarity: how similar the composition of species is to an intact baseline

The collection we will be working with is `io-biodiversity`. 

**2. Phoenix Subdivison** 

The second dataste is from the [United Census Bureau](https://www.census.gov/cgi-bin/geo/shapefiles/). This data has been downloaded as a shapefile and has 19 columns. It contains the county names, land area and water area to name a few.    

## Final Output

The final visualization

### Import Libraries 

In [5]:
# Import Libraries
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

# Used to access STAC catalogs
#from pystac_client import Client
# Used to sign items from the MPC STAC catalog
#import planetary_computer

# other libraries for nice outputs
# from IPython.display import Image

# Package to work with web-tiles for background maps
import contextily as cx

ModuleNotFoundError: No module named 'contextily'

### Catalog Search

We can narrow the search within the `catalog` by specifying our
- Time range: 2017 - 2020
- Area of Interest: `[-112.826843, 32.974108, -111.184387, 33.863574]`
- Collection name: `io-biodiversity`

In [2]:
# Data Import: BII data

## Acess Planatery Computer Catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## Catalog Search
# Set temporal range
time_range = "2017/2020"
# Bonding box of Phoenix subdivision
bbox = {
   "type": "Polygon",
    "coordinates": [
        [
            [-112.826843, 32.974108],
            [-111.184387, 32.974108],
            [-111.184387, 33.863574],
            [-112.826843, 33.863574],
            [-112.826843, 32.974108]  # Make sure the first and last coordinates are the same to close the polygon
        ]
    ]
}
# catalog search
search = catalog.search(
    collections=['io-biodiversity'],
    intersects=bbox,
    datetime = time_range
)

# Get items found in catalog search
item = search.item_collection()

# get first item in the catalog search
item = items[0]

NameError: name 'Client' is not defined

Lets view the items in the catalog search and select the first item!

In [ ]:
# Get items found in catalog search
items = search.item_collection()
​
# get first item in the catalog search
item = items[0]


Now we can view the item assets keys. Our assets of interest are:

- `data`: which contains the data we are interested in
- `rendered_preview`: a preview of our raster imagery

In [ ]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

Let's view our rendered preview.

In [ ]:
# plot rendered preview
Image(url=item.assets['rendered_preview'].href, width=500)

### Import Data

Now we can select our `data` from the catelog for our analysis  

In [ ]:
maricopa = rioxr.open_rasterio(item.assets['data'].href)
maricopa

#### Data Import: Arizona Counties shapefile 

In [6]:
# Data Import: Arizona counties shapefile
arizona_counties = gpd.read_file('data/tl_2022_04_cousub/tl_2022_04_cousub.shp')

# Data Import: U.S. states shapefile
us_states = gpd.read_file('data/states_shapefile/States_shapefile.shp')

## Prepare Data

In [9]:
# Lets make this lower case 
arizona_counties.columns = arizona_counties.columns.str.lower()

# Filter to Maricopa County
maricopa_counties = arizona_counties[arizona_counties.name =='Maricopa-Stanfield']

# Let's view our results 
maricopa_counties

,statefp,countyfp,cousubfp,cousubns,geoid,name,namelsad,lsad,classfp,mtfcc,cnectafp,nectafp,nctadvfp,funcstat,aland,awater,intptlat,intptlon,geometry
57,04,021,92091,01934958,0402192091,Maricopa-Stanfield,Maricopa-Stanfield CCD,22,Z5,G4040,None,None,None,S,1026963163,668887,+32.9120190,-112.0749104,"POLYGON ((-112.20357 32.68926, -112.20357 32.6..."
